In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
from data_quality import cargue_datasets_c
casos, edu_vial2018, hurto_tp, mede_victimas, compar = cargue_datasets_c()

In [3]:
casos

,Año,Conducta,Comuna,Cantidad_casos
0,2003,Extorsión,3.0,1
1,2004,Extorsión,10.0,2
2,2004,Extorsión,13.0,1
3,2004,Extorsión,16.0,2
4,2004,Extorsión,4.0,1
...,...,...,...,...
2365,2021,Hurto de semoviente,60.0,2
2366,2021,Hurto de semoviente,80.0,2
2367,2022,Hurto de semoviente,50.0,1
2368,2022,Hurto de semoviente,60.0,1


In [4]:
# Supongamos que tienes dos DataFrames df1 y df2 con una columna clave común "ID"
df = pd.merge(casos, edu_vial2018, on=["Comuna","Año"], how="inner")
df = pd.merge(df, hurto_tp, on=["Comuna","Año"], how="inner")
#df = pd.merge(df, mede_victimas, on=["Comuna","Año"], how="inner")
#df = pd.merge(df, compar, on=["Comuna","Año"], how="inner")
df

,Año,Conducta_x,Comuna,Cantidad_casos,Fecha_x,Acciones,Publico,N_personas,N_mujeres,N_hombres,...,N_hombres_ac/pas,Fecha_y,Sexo,Edad,Estado_civil,Transporte,Modalidad,Conducta_y,Lugar,Bien
0,2019,Extorsión,1.0,14,2019-09-25,Socializacion_Proyectos_Movilidad_Conglomerado...,Personal Administrativo ó Gerencial,1,0,1,...,0,2019-01-20,Mujer,24,Soltero(a),Metro,Cosquilleo,De celular,Estación del Metro,Celular
1,2019,Extorsión,1.0,14,2019-09-25,Socializacion_Proyectos_Movilidad_Conglomerado...,Personal Administrativo ó Gerencial,1,0,1,...,0,2019-04-03,Mujer,19,Unión marital de hecho,Taxi,Engaño,De celular,Vía pública,Peso
2,2019,Extorsión,1.0,14,2019-09-25,Socializacion_Proyectos_Movilidad_Conglomerado...,Personal Administrativo ó Gerencial,1,0,1,...,0,2019-04-03,Mujer,19,Unión marital de hecho,Taxi,Engaño,De celular,Vía pública,Accesorios prendas de vestir
3,2019,Extorsión,1.0,14,2019-09-25,Socializacion_Proyectos_Movilidad_Conglomerado...,Personal Administrativo ó Gerencial,1,0,1,...,0,2019-04-03,Mujer,19,Unión marital de hecho,Taxi,Engaño,De celular,Vía pública,Celular
4,2019,Extorsión,1.0,14,2019-09-25,Socializacion_Proyectos_Movilidad_Conglomerado...,Personal Administrativo ó Gerencial,1,0,1,...,0,2019-04-29,Mujer,28,Unión marital de hecho,Autobus,Raponazo,Sin dato,Metro Plus,Celular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7120939,2019,Hurto a residencia,90.0,113,2019-04-08,Otros_Instituciones_Educativas,Niños y Niñas de 6 a 14 Años,183,97,86,...,0,2019-10-06,Mujer,33,Soltero(a),Autobus,Cosquilleo,Sin dato,Paradero de bus,Elementos computador
7120940,2019,Hurto a residencia,90.0,113,2019-04-08,Otros_Instituciones_Educativas,Niños y Niñas de 6 a 14 Años,183,97,86,...,0,2019-10-06,Mujer,33,Soltero(a),Autobus,Cosquilleo,Sin dato,Paradero de bus,Cédula
7120941,2019,Hurto a residencia,90.0,113,2019-04-08,Otros_Instituciones_Educativas,Niños y Niñas de 6 a 14 Años,183,97,86,...,0,2019-10-20,Mujer,49,Soltero(a),Autobus,Descuido,Sin dato,Vehículo particular,Celular
7120942,2019,Hurto a residencia,90.0,113,2019-04-08,Otros_Instituciones_Educativas,Niños y Niñas de 6 a 14 Años,183,97,86,...,0,2019-12-10,Mujer,27,Soltero(a),Taxi,Descuido,Sin dato,Restaurante,Cédula


In [5]:
df = df.drop(df.columns[[4]], axis=1)